# Задача классификации графов, предсказание свойств горения молекул углеводорода
## Постановка задачи: предсказать один из индикаторов качества горения – производное цетановое число (DCN) для оксигенированных углеводородов по структуре молекул.

In [13]:
import sys
sys.path.append('../')
import torch_geometric.transforms as T
import torch
import pandas as pd

from stable_gnn.pipelines.graph_classification_pipeline import TrainModelGC, TrainModelOptunaGC
from stable_gnn.graph import Graph

# Загрузка датасета, состоящего из молекул.
 Атрибуты вершин: относится ли атом к определенному типу (например C, N, S и т.д.), степень вершин-атомов, формальный заряд атома, тип гибридизации, является ли атом частью кольца, является ли атом частью ароматического соединения, нормированная атомная масса

In [14]:
root = '../data_validation/'
name='fuel'
dataset = Graph(root=root + str(name), name=name, transform=T.NormalizeFeatures(),adjust_flag=False)
len(dataset)

236

## Решаем задачу предсказания связей, пользуясь подготовленным пайплайном в stable_gnn.pipelines.train_model_pipeline
Задаем различные конфигурации включения экстраполяции и самостоятельного обучения

In [15]:
results = pd.DataFrame(columns=['extrapolate_flag', 'ssl_flag','test accuracy']) # табличка c результатми решения задачи для различных конфигураций параметров включения самостоятельного обучения и экстраполяции

In [16]:
conv = "GAT"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ssl_flag = False
extrapolate_flag = False

    #######

optuna_training = TrainModelOptunaGC(
        data=dataset,
            conv=conv,
            device=device,
            ssl_flag=ssl_flag,
            extrapolate_flag=extrapolate_flag,
        )

best_values = optuna_training.run(number_of_trials=50)

model_training = TrainModelGC(
            data=dataset,
            conv=conv,
            device=device,
            ssl_flag=ssl_flag,
            extrapolate_flag=extrapolate_flag,
)

model, train_acc_mi, train_acc_ma, test_acc_mi, test_acc_ma = model_training.run(best_values)
print(test_acc_mi)
results=results.append(pd.Series([extrapolate_flag,ssl_flag,test_acc_mi],index=results.columns), ignore_index=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0.5416666666666666


In [17]:
results

,extrapolate_flag,ssl_flag,test accuracy
0,False,False,0.541667


## Extrapolate_flag = True, ssl_flag = False

In [18]:
conv = "GAT"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ssl_flag = False
extrapolate_flag = True

    #######


optuna_training = TrainModelOptunaGC(
        data=dataset,
            conv=conv,
            device=device,
            ssl_flag=ssl_flag,
            extrapolate_flag=extrapolate_flag,
        )

best_values = optuna_training.run(number_of_trials=50)

model_training = TrainModelGC(
            data=dataset,
            conv=conv,
            device=device,
            ssl_flag=ssl_flag,
            extrapolate_flag=extrapolate_flag,
)

model, train_acc_mi, train_acc_ma, test_acc_mi, test_acc_ma = model_training.run(best_values)
print(test_acc_mi)
results=results.append(pd.Series([extrapolate_flag,ssl_flag,test_acc_mi],index=results.columns), ignore_index=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0.725


## Extrapolate_flag = False, ssl=True


In [19]:
conv = "GAT"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ssl_flag = True
extrapolate_flag = False

    #######
optuna_training = TrainModelOptunaGC(
        data=dataset,
            conv=conv,
            device=device,
            ssl_flag=ssl_flag,
            extrapolate_flag=extrapolate_flag,
        )

best_values = optuna_training.run(number_of_trials=50)

model_training = TrainModelGC(
            data=dataset,
            conv=conv,
            device=device,
            ssl_flag=ssl_flag,
            extrapolate_flag=extrapolate_flag,
)

model, train_acc_mi, train_acc_ma, test_acc_mi, test_acc_ma = model_training.run(best_values)
print(test_acc_mi)
results=results.append(pd.Series([extrapolate_flag,ssl_flag,test_acc_mi],index=results.columns), ignore_index=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0.8166666666666668


## Extrapolate_flag = True, ssl_flag = True

In [20]:
conv = "GAT"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ssl_flag = True
extrapolate_flag = True

optuna_training = TrainModelOptunaGC(
        data=dataset,
            conv=conv,
            device=device,
            ssl_flag=ssl_flag,
            extrapolate_flag=extrapolate_flag,
        )

best_values = optuna_training.run(number_of_trials=50)

model_training = TrainModelGC(
            data=dataset,
            conv=conv,
            device=device,
            ssl_flag=ssl_flag,
            extrapolate_flag=extrapolate_flag,
)

model, train_acc_mi, train_acc_ma, test_acc_mi, test_acc_ma = model_training.run(best_values)
print(test_acc_mi)
results=results.append(pd.Series([extrapolate_flag,ssl_flag,test_acc_mi],index=results.columns), ignore_index=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0.8250000000000001


In [21]:
results

,extrapolate_flag,ssl_flag,test accuracy
0,False,False,0.541667
1,True,False,0.725
2,False,True,0.816667
3,True,True,0.825
